In [1]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.models import model_from_json
import re
from keras.callbacks import TensorBoard
from time import time

from sklearn.preprocessing import LabelEncoder

data = pd.read_csv('Sentiment.csv')
# Keeping only the neccessary columns
data = data[['text', 'sentiment']]
data = data[data.sentiment != 'Neutral']

data = data[data.sentiment != "Neutral"]
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

print(data[data['sentiment'] == 'Positive'].size)
print(data[data['sentiment'] == 'Negative'].size)

for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ')

max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
#print(X)
X = pad_sequences(X)
#print(X)
embed_dim = 128
lstm_out = 196


def createmodel():
    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim, input_length=X.shape[1]))
    model.add(SpatialDropout1D(0.4))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


# print(model.summary())

labelencoder = LabelEncoder()
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.33, random_state=42)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

tensorborad = TensorBoard(log_dir="logs/{}".format(time()))

batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, epochs=5, batch_size=batch_size, verbose=2)
score, acc = model.evaluate(X_test, Y_test, verbose=2, batch_size=batch_size)
model.fit(X_train[0:2000], Y_train[0:2000], epochs=5, validation_data=(X_test[0:2000], Y_test[0:2000]), callbacks=[tensorborad])
print(score)
print(acc)

# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("model saved")

# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("model loaded")

# evaluate loaded model on test data
loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
score = loaded_model.evaluate(X, y, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1] * 100))

twt = ['A lot of good things are happening. We are respected again throughout the world, and thats a great thing.@realDonaldTrump']

# vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)

# padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt, batch_size=1, verbose=2)[0]
print(sentiment)

if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

Using TensorFlow backend.


4472
16986
(7188, 28) (7188, 2)
(3541, 28) (3541, 2)


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
 - 17s - loss: 0.4408 - accuracy: 0.8183
Epoch 2/5
 - 15s - loss: 0.3256 - accuracy: 0.8637
Epoch 3/5
 - 15s - loss: 0.2780 - accuracy: 0.8854
Epoch 4/5
 - 15s - loss: 0.2450 - accuracy: 0.9028
Epoch 5/5
 - 15s - loss: 0.2219 - accuracy: 0.9111
Train on 2000 samples, validate on 2000 samples
Epoch 1/5
2000/2000 [==============================] - 5s 3ms/step - loss: 0.1910 - accuracy: 0.9225 - val_loss: 0.3965 - val_accuracy: 0.8360
Epoch 2/5
2000/2000 [==============================] - 5s 2ms/step - loss: 0.1630 - accuracy: 0.9385 - val_loss: 0.4370 - val_accuracy: 0.8355
Epoch 3/5
2000/2000 [==============================] - 5s 2ms/step - loss: 0.1294 - accuracy: 0.9570 - val_loss: 0.4720 - val_accuracy: 0.8435
Epoch 4/5
2000/2000 [==============================] - 5s 2ms/step - loss: 0.1067 - accuracy: 0.9635 - val_loss: 0.5216 - val_accuracy: 0.8280
Epoch 5/5
2000/2000 [==============================] - 5s 2ms/step - loss: 0.0960 - accuracy: 0.9690 - val_loss: 0.5618 - val